
# 🚀 Proyecto Hackathon — Pipeline ETL Documentado (Google Colab)

## 📌 Resumen Ejecutivo

Este cuaderno documenta de manera **profesional, reproducible y explicada paso a paso** un proceso completo de **ETL (Extract, Transform, Load)** aplicado a un dataset de clientes.

### 🎯 Objetivos del ETL
- **Estandarizar** nombres de columnas y valores categóricos.
- **Limpiar datos**: eliminar caracteres especiales, normalizar texto y corregir errores conocidos.
- **Imputar valores faltantes** de forma controlada.
- **Validar calidad de datos** mediante reglas de negocio:
  - Rangos permitidos.
  - Variables binarias válidas.
  - Ausencia de nulos.
  - Ausencia de duplicados.
- **Transformar variables** para su correcto análisis:
  - Conversión a binarios.
  - Redondeo de decimales.
  - Escalado de métricas.
- **Generar un dataset final confiable** listo para análisis o modelado.

Este enfoque permite:

✔️ Trazabilidad de cada transformación.  
✔️ Reproducibilidad del proceso.  
✔️ Auditoría de calidad de datos.  
✔️ Escalabilidad para futuros datasets.

---

🔎 Nota sobre la ejecución del ETL

Este cuaderno incluye, al final, una celda con el pipeline completo del proceso ETL consolidado.
Para garantizar resultados consistentes y evitar efectos acumulados de ejecuciones parciales, se recomienda ejecutar únicamente esa celda final al momento de generar el dataset definitivo.


## 📦 Importación de librerías

In [42]:
import pandas as pd
import numpy as np
import unicodedata
import re


## 🔹 ETAPA 1 — EXTRACCIÓN

Se carga el archivo CSV original en un DataFrame de pandas.  
Esta etapa únicamente **lee los datos sin modificarlos**.


In [58]:
df = pd.read_csv('/content/customer_dataset_consistente.csv', sep=";")

In [59]:
df

,cliente_id,genero,edad,pais,ciudad,segmento_de_cliente,meses_permanencia,canal_de_registro,tipo_contrato,conecciones_mensuales,...,tiempo_promedio_de_resolucion,tipo_de_queja,puntuacion_csates,escaladas,tasa_apertura_email,puntuacion_nps,respuesta_de_la_encuesta,Estado de Conexión,Permanencia Corta,abandonar
0,CUST_00001,Masculino,68,UK,Londres,SME,48,Web,Mensual,16,...,13.354.359.704.932.100,Facturacion,4.0,0,0.880,27,Satisfecho,En Riesgo,0,0
1,CUST_00002,Femenino,57,Australia,Sydney,Individual,6,Mobile,Mensual,1,...,2.514.008.841.258.960,Facturacion,2.0,0,0.027,-40,Insatisfecho,Inactivo,1,1
2,CUST_00003,Masculino,24,US,New York,SME,58,Web,Anual,19,...,2.757.292.845.215.130,Facturacion,3.0,0,0.350,0,Neutral,En Riesgo,0,0
3,CUST_00004,Masculino,49,Banglades,Dhaka,Individual,17,Mobile,Anual,10,...,26.420.822.304.921.700,Tecnico,5.0,1,0.895,100,Muy satisfecho,En Riesgo,0,0
4,CUST_00005,Masculino,65,India,Delhi,Individual,49,Web,Mensual,6,...,26.674.579.176.744.200,Tecnico,4.0,0,0.921,21,Satisfecho,En Riesgo,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,CUST_09996,Femenino,67,Australia,Sydney,SME,31,Web,Anual,8,...,3.424.017.394.431.900,Facturacion,4.0,1,0.819,20,Satisfecho,Activo,0,0
9996,CUST_09997,Masculino,26,Canada,Toronto,Individual,22,Mobile,Anual,14,...,40.092.087.117.431.100,Facturacion,4.0,0,0.822,28,Satisfecho,En Riesgo,0,0
9997,CUST_09998,Femenino,18,Australia,Sydney,Individual,1,Mobile,Anual,1,...,25.215.809.846.857.700,Facturacion,1.0,0,0.105,-80,Muy insatisfecho,Inactivo,1,1
9998,CUST_09999,Femenino,30,India,Delhi,Individual,1,Web,Mensual,0,...,20.544.921.075.268.500,Facturacion,2.0,0,0.013,-40,Insatisfecho,Inactivo,1,1



## 🔹 ETAPA 2 — Normalización de Encabezados

En esta sección se estandarizan los encabezados del dataset para asegurar consistencia, legibilidad y compatibilidad durante el procesamiento de datos.

La función limpiar_columna() aplica una limpieza automática a cada nombre de columna, realizando las siguientes transformaciones:

Eliminación de acentos y caracteres especiales mediante normalización Unicode.

Conversión de todo el texto a minúsculas.

Reemplazo de espacios por guiones bajos (_).

Eliminación de símbolos no alfanuméricos.

Posteriormente, esta función se aplica a todos los encabezados del dataset mediante una comprensión de listas, garantizando que todas las columnas sigan el mismo estándar de nomenclatura.

Finalmente, se ejecuta un bloque de correcciones manuales que permite ajustar errores conocidos en nombres de columnas (por ejemplo, errores ortográficos), asegurando uniformidad semántica en los campos.


In [60]:
def limpiar_columna(nombre):
    # Quita acentos
    nombre = unicodedata.normalize('NFKD', nombre)
    nombre = nombre.encode('ascii', 'ignore').decode('utf-8')

    # Convierte a minúsculas
    nombre = nombre.lower().strip()

    # Reemplazar espacios por _
    nombre = re.sub(r'\s+', '_', nombre)

    # Eliminar caracteres especiales
    nombre = re.sub(r'[^a-z0-9_]', '', nombre)

    return nombre

# Aplicar limpieza automática
df.columns = [limpiar_columna(col) for col in df.columns]

# Correcciones manuales de errores conocidos
correcciones = {
    'conecciones_mensuales': 'conexiones_mensuales',
    'promedio_coneccion': 'promedio_conexion',
    'ultima_coneccion': 'ultima_conexion'

}

for col_mal, col_bien in correcciones.items():
    if col_mal in df.columns:
        df.rename(columns={col_mal: col_bien}, inplace=True)



## 🔹 ETAPA 3 — Funciones Auxiliares

En esta etapa se realiza la limpieza y estandarización de todas las columnas de tipo texto del dataset con el objetivo de mejorar la calidad de los datos y evitar inconsistencias durante el análisis.

La función limpiar_texto() aplica las siguientes transformaciones a cada valor textual:

Eliminación de acentos y caracteres especiales mediante normalización Unicode.

Conservación únicamente de caracteres alfanuméricos y espacios.

Eliminación de espacios duplicados y espacios al inicio o al final del texto.

Preservación de valores nulos para no alterar registros incompletos.

Posteriormente, se identifican todas las columnas de tipo texto (object) y se convierten todos sus valores a minúsculas, garantizando uniformidad en la escritura y evitando duplicidades causadas por diferencias de capitalización.

Este proceso asegura que los campos textuales sean consistentes, comparables y más fáciles de analizar o categorizar.

In [61]:
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    # Quitar acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    # Quitar caracteres especiales
    texto = re.sub(r'[^A-Za-z0-9\s]', '', texto)
    # Quitar espacios duplicados
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# ---------- Convertir todo el texto a minúsculas ----------
columnas_texto = df.select_dtypes(include=['object']).columns
for col in columnas_texto:
    df[col] = df[col].str.lower()


## 🔹 ETAPA 4 — TRANSFORMACIONES



### ✔️ Normalización de tipo_de_queja
Para la normalización de la variable tipo_de_queja, se definieron reglas de negocio orientadas a garantizar coherencia semántica entre la cantidad de tickets registrados y la clasificación de la queja.

Reglas:
- Si tickets_de_soporte == 0 → "sin queja"

  - Cuando el valor de tickets_de_soporte es igual a cero, se interpreta que el cliente no ha reportado ningún incidente, por lo que la categoría se asigna automáticamente como “sin queja”, independientemente del valor previo del campo.

- Si es nulo y tickets == 0 → "sin queja"

  - En los casos donde tipo_de_queja presenta valores nulos y el número de tickets es cero, se refuerza esta misma clasificación para evitar ambigüedades.

- Si es nulo y tickets > 0 → "otro tipo de queja"

  - Cuando el campo tipo_de_queja es nulo y existe al menos un ticket de soporte registrado, el registro se clasifica como “otro tipo de queja”, permitiendo conservar la información de que existe una incidencia aun cuando el detalle específico no esté disponible.

Estas reglas aseguran consistencia lógica, reducen valores faltantes y mejoran la calidad del dataset para análisis posteriores.

In [62]:
# - Si tickets_de_soporte == 0 → "sin queja"
df.loc[df['tickets_de_soporte'] == 0, 'tipo_de_queja'] = "sin queja"

# - Si tickets_de_soporte == 0 y tipo_queja es null → "sin queja"
df.loc[(df['tipo_de_queja'].isnull()) & (df['tickets_de_soporte'] == 0), 'tipo_de_queja'] = "sin queja"

# - Si tickets_de_soporte > 0 → "Otra queja"
df.loc[(df['tipo_de_queja'].isnull()) & (df['tickets_de_soporte'] > 0), 'tipo_de_queja'] = "otro tipo de queja"

# Reporte después de la corrección
nulos_final = df['tipo_de_queja'].isnull().sum()

print("\n📊 RESUMEN FINAL DEL ETL")
print(f"Nulos restantes en tipo_de_queja: {nulos_final}")



📊 RESUMEN FINAL DEL ETL
Nulos restantes en tipo_de_queja: 0



### ✔️ Conversión de Variables Binarias


Este bloque de código identifica las columnas que representan variables binarias (descuento_aplicado y aumento_ultimos_3_meses) y las estandariza para que contengan únicamente valores numéricos 0 y 1.

Primero, los valores de cada columna se convierten a texto, se eliminan espacios innecesarios y se transforman a minúsculas para evitar inconsistencias de formato (por ejemplo: "Sí", " si ", "1").

Posteriormente, se utiliza un diccionario de mapeo para convertir los valores textuales a valores numéricos:

"si", "sí" y "1" → 1

"no" y "0" → 0

Los valores que no coinciden con ninguno de los casos definidos se consideran inválidos y se reemplazan automáticamente por 0, garantizando que la columna quede completamente numérica y lista para análisis o modelado.

Finalmente, los datos se convierten explícitamente a tipo entero (int) para asegurar consistencia y facilitar validaciones posteriores.

In [63]:
# ---------- Variables binarias ----------
binarias = ['descuento_aplicado', 'aumento_ultimos_3_meses']

mapa_binario = {
    'si': 1,
    'sí': 1,
    'no': 0,
    '1': 1,
    '0': 0
}

for col in binarias:
    if col in df.columns:
        normalizado = (
            df[col]
            .astype(str)
            .str.strip()
            .str.lower()
        )

        df[col] = normalizado.map(mapa_binario)

        # Valores no reconocidos → 0
        df[col] = df[col].fillna(0).astype(int)

        df[col] = df[col].fillna(0).astype(int)


### ✔️ Redondeo Automático de Columnas Numéricas


Este bloque identifica automáticamente las columnas numéricas que contienen valores con más de dos decimales y aplica un redondeo únicamente a aquellas que lo requieren.

Primero, se seleccionan todas las columnas de tipo numérico del dataset. Para cada columna, se analiza la parte decimal de sus valores (ignorando nulos) y se verifica si existe al menos un dato con más de dos cifras decimales.

La lógica evalúa:

- La parte fraccionaria de cada valor numérico.

- El número de dígitos decimales significativos.

- Si algún valor supera los dos decimales permitidos.

Solo las columnas que cumplen esta condición son agregadas a la lista columnas_a_redondear y posteriormente redondeadas a dos decimales. Esto evita modificar columnas que ya cumplen con el nivel de precisión deseado.

Este enfoque garantiza:

- Consistencia en la precisión numérica.

- Preservación de datos que no requieren ajuste.

- Mejor control de calidad para análisis y reportes.

In [64]:
# ---------- Detectar columnas con más de 2 decimales ----------
columnas_numericas = df.select_dtypes(include=[np.number]).columns
columnas_a_redondear = []

for col in columnas_numericas:
    tiene_muchos_decimales = (
        (df[col].dropna() % 1).round(10)
        .apply(lambda x: len(str(x).split('.')[-1].rstrip('0')) > 2)
        .any()
    )

    if tiene_muchos_decimales:
        columnas_a_redondear.append(col)
        df[col] = df[col].round(2)

print("Columnas redondeadas:", columnas_a_redondear)

Columnas redondeadas: ['tasa_apertura_email']



### ✔️ Limpieza Final de Texto


En esta etapa se aplican reglas de limpieza a todas las columnas de tipo texto del dataset utilizando la función limpiar_texto().

El proceso identifica automáticamente las columnas categóricas (object) y transforma cada uno de sus valores para:

- Eliminar acentos y normalizar caracteres Unicode.

- Remover caracteres especiales no deseados.

- Reducir múltiples espacios a un solo espacio.

- Eliminar espacios al inicio y al final del texto.

- Conservar valores nulos sin alterarlos.

Este procedimiento permite estandarizar la información textual, reducir ruido en los datos y evitar errores en análisis posteriores, agrupaciones o validaciones.

In [65]:
# ---------- Limpieza de texto ----------
columnas_texto = df.select_dtypes(include=['object']).columns
for col in columnas_texto:
    df[col] = df[col].apply(limpiar_texto)


### ✔️ Escalado de puntuación NPS


Este bloque transforma la variable puntuacion_nps desde su escala original de -100 a 100 hacia una nueva escala normalizada de 0 a 10, facilitando su interpretación y uso en análisis posteriores.

Para ello, se define la función escalar(x), la cual aplica una transformación lineal que preserva la proporción entre los valores originales:

-100 → 0

0 → 5

100 → 10

Posteriormente, la función se aplica a toda la columna puntuacion_nps y los resultados se redondean al entero más cercano para simplificar su análisis.

Finalmente, se ejecuta una validación automática que verifica que todos los valores transformados se encuentren dentro del rango permitido (0 a 10). Si se detecta algún valor fuera de este rango, el proceso se detiene y se genera un error, garantizando la calidad y consistencia de los datos.

In [66]:
# ---------- Validación de rango de puntuación ----------
def escalar(x):
    """
    Convierte valores de rango [-100, 100] a [0, 10]
    """
    return ((x + 100) * 10 / 200)

# ---------- Escalado de puntuación ----------

# Cambia 'puntuacion' si tu columna tiene otro nombre
if 'puntuacion_nps' in df.columns:
  df['puntuacion_nps'] = df['puntuacion_nps'].apply(escalar).round(0)

if 'puntuacion_nps' in df.columns:
    assert df['puntuacion_nps'].between(0, 10).all(), \
        "Existen valores fuera del rango 0–10 en puntuacion_nps"



### ✔️ Normalización de Categorías


Este bloque estandariza valores inconsistentes en columnas categóricas clave para evitar duplicidades semánticas y mejorar la calidad del dataset.

En la columna metodo_de_pago, se detectan registros que contienen la variante "transferencia bancarias" (en plural o con posibles espacios) y se normalizan al valor único "transferencia bancaria".

De manera similar, en la columna canal_de_registro, los registros con el valor "referral" se reemplazan por "referido", unificando el idioma y manteniendo coherencia en las categorías.

Para ambos casos, se aplica una comparación robusta que:

* Elimina espacios al inicio y al final del texto.

* Convierte los valores a minúsculas antes de comparar.

Este proceso mejora la confiabilidad de análisis, segmentaciones y reportes.

In [67]:
# ---------- Normalización de categorías ----------
if 'metodo_de_pago' in df.columns:
    mask = df['metodo_de_pago'].str.strip().str.lower() == 'transferencia bancarias'
    df.loc[mask, 'metodo_de_pago'] = 'transferencia bancaria'

if 'canal_de_registro' in df.columns:
    mask = df['canal_de_registro'].str.strip().str.lower() == 'referral'
    df.loc[mask, 'canal_de_registro'] = 'referido'


## 🔹 ETAPA 5 — VALIDACIONES DE CALIDAD


Este bloque realiza una verificación de calidad para identificar la presencia de valores nulos en el dataset final.

Primero, se calcula el total de valores nulos por cada columna. Posteriormente, se filtran únicamente aquellas columnas que presentan al menos un valor nulo.

Si existen columnas con valores faltantes, el sistema imprime un reporte indicando qué variables contienen datos incompletos y cuántos registros están afectados. En caso contrario, se muestra un mensaje confirmando que el dataset no contiene valores nulos.

In [68]:
# =====================================================
# VALIDACIONES DE CALIDAD
# =====================================================

# --- Sin nulos ---
nulos = df.isnull().sum()
nulos = nulos[nulos > 0]

if len(nulos) > 0:
    print("⚠️ Nulos por columna:")
    print(nulos)
else:
    print("No existen valores nulos en el dataset")

No existen valores nulos en el dataset


Este bloque implementa una estrategia de corrección automática para los valores faltantes detectados en el dataset.

Para cada columna que contiene valores nulos:

Si la columna es numérica (int64 o float64), los valores faltantes se reemplazan por la mediana de la columna. Esta medida es robusta frente a valores extremos y preserva la distribución de los datos.

Si la columna es categórica o de texto, los valores nulos se reemplazan por el valor "No informado", permitiendo mantener la integridad del dataset sin perder registros.

Este enfoque permite:

* Evitar errores en análisis posteriores o procesos de carga.

* Mantener consistencia en la estructura de los datos.

* Reducir la pérdida de información causada por eliminación de registros.

In [69]:
# Corrección automática de nulos
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype in ['int64', 'float64']:
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = df[col].fillna('No informado')

Este bloque verifica que los valores de la variable puntuacion_csates se encuentren dentro del rango permitido de 1 a 5, de acuerdo con la escala definida para la medición de satisfacción.

Para ello:

Se crea una columna auxiliar llamada csates_invalido, que marca con:

* 1 los registros cuya puntuación está fuera del rango válido (menor a 1 o mayor a 5).

* 0 los registros que cumplen con el rango esperado.

A partir de esta marca, se genera un reporte (reporte_csates) que muestra únicamente los registros inconsistentes, incluyendo el identificador del cliente, la puntuación registrada y la respuesta asociada.

Si existen inconsistencias, estas se imprimen para su revisión. En caso contrario, se informa que no se detectaron valores fuera de rango.

Luego se calcula el total de registros que presentan inconsistencias en la puntuación CSAT, sumando los valores de la columna indicadora csates_invalido. Este conteo permite conocer rápidamente el nivel de calidad de la variable evaluada.

Se elimina la columna auxiliar csates_invalido, ya que su único propósito fue apoyar el proceso de validación y no es necesaria para el dataset final.

Finalmente, se muestra un mensaje confirmando qué columnas fueron eliminadas, manteniendo trazabilidad y claridad sobre las transformaciones aplicadas.

In [70]:
# Validar rango permitido (1–5)
df['csates_invalido'] = df['puntuacion_csates'].apply(lambda x: 1 if x < 1 or x > 5 else 0)

# Reporte de inconsistencias
reporte_csates = df[df['csates_invalido'] == 1][['cliente_id', 'puntuacion_csates', 'respuesta_de_la_encuesta']]

if len(reporte_csates) > 0:
    print("⚠️ Registros con puntuacion_csates fuera de rango:")
    print(reporte_csates)
else:
    print("No existen inconsistencias en puntuacion_csates")

# Conteo total
print(f"Total de inconsistencias en CSAT: {df['csates_invalido'].sum()}")

columnas_remover=[
'csates_invalido'
]
df = df.drop(columnas_remover, axis=1, errors='ignore')
print(f"Columnas eliminadas: {', '.join(columnas_remover)}")



No existen inconsistencias en puntuacion_csates
Total de inconsistencias en CSAT: 0
Columnas eliminadas: csates_invalido


Este bloque ejecuta una serie de validaciones finales para garantizar que el dataset resultante cumpla con los criterios mínimos de calidad antes de ser exportado.

Las validaciones realizadas son:

* Ausencia de valores nulos

Se verifica que no existan valores faltantes en ninguna columna del dataset. Si se detecta algún valor nulo, el proceso se detiene y se genera un error, evitando la propagación de datos incompletos.

* Consistencia de variables binarias

Se valida que las columnas definidas como binarias contengan únicamente valores permitidos (0 y 1). Esto asegura que las variables estén correctamente codificadas y listas para análisis o modelado.

* Ausencia de registros duplicados

Se comprueba que no existan filas duplicadas en el dataset final, garantizando unicidad y evitando sesgos en análisis posteriores.

* Dataset no vacío

Se valida que el dataset contenga al menos un registro. Esto previene errores derivados de cargas vacías o fallos en el proceso de extracción o transformación.

Si cualquiera de estas condiciones no se cumple, el ETL se interrumpe automáticamente, lo que protege la integridad de los datos y facilita la detección temprana de errores.

In [71]:
# Validación final
assert df.isnull().sum().sum() == 0, "Existen valores nulos en el dataset final"

# --- Binarios válidos ---
for col in binarias:
    if col in df.columns:
        assert set(df[col].dropna().unique()) <= {0, 1}, \
            f"Valores inválidos en {col}"

# --- Sin duplicados ---
assert df.duplicated().sum() == 0, "Existen registros duplicados"

# --- Dataset no vacío ---
assert df.shape[0] > 0, "El dataset final está vacío"


## 🔹 ETAPA 6 — CARGA

En esta etapa se realiza la exportación del dataset ya transformado y validado a un archivo CSV llamado customer_dataset_etl.csv.

El archivo se genera sin incluir el índice del DataFrame y utilizando codificación UTF-8, garantizando compatibilidad con herramientas de análisis, hojas de cálculo y sistemas externos.

Finalmente, se imprime un mensaje de confirmación que indica la correcta finalización del proceso ETL, permitiendo al usuario validar visualmente que el pipeline se ejecutó sin errores.

Este paso representa el cierre del flujo de procesamiento y deja el dataset listo para su consumo en análisis, reportes o integraciones posteriores.

In [72]:
# =====================================================
# LOAD
# =====================================================
df.to_csv(
    'customer_dataset_etl.csv',
    index=False,
    encoding='utf-8'
)

print("========================================")
print("✅ ETL ejecutado correctamente.")

✅ ETL ejecutado correctamente.


# Ejecución final del ETL completo (versión consolidada)

En este apartado se ejecuta el pipeline completo de ETL en un solo bloque, integrando todas las validaciones, normalizaciones y correcciones aplicadas previamente.

El objetivo de este bloque es:

* Garantizar consistencia en los tipos de datos.

* Aplicar todas las transformaciones de forma ordenada.

* Validar la calidad final del dataset.

* Exportar el archivo limpio y listo para análisis.

Para obtener el resultado correcto, únicamente carga el archivo customer_datase_consistente.csv y ejecuta únicamente esta celda.

In [4]:
import pandas as pd
import numpy as np
import unicodedata
import re

# =====================================================
# EXTRACT
# =====================================================
df = pd.read_csv('/content/customer_dataset_consistente.csv')

# =====================================================
# NORMALIZACIÓN DE ENCABEZADOS
# =====================================================

def limpiar_columna(nombre):
    # Normalizar caracteres raros / acentos
    nombre = unicodedata.normalize('NFKD', nombre)
    nombre = nombre.encode('ascii', 'ignore').decode('utf-8')

    # Pasar a minúsculas
    nombre = nombre.lower().strip()

    # Reemplazar espacios por _
    nombre = re.sub(r'\s+', '_', nombre)

    # Eliminar caracteres especiales
    nombre = re.sub(r'[^a-z0-9_]', '', nombre)

    return nombre

# Aplicar limpieza automática
df.columns = [limpiar_columna(col) for col in df.columns]

# Correcciones manuales de errores conocidos
correcciones = {
    'conecciones_mensuales': 'conexiones_mensuales',
    'promedio_coneccion': 'promedio_conexion',
    'ultima_coneccion': 'ultima_conexion'

}

for col_mal, col_bien in correcciones.items():
    if col_mal in df.columns:
        df.rename(columns={col_mal: col_bien}, inplace=True)


# =====================================================
# FUNCIONES AUXILIARES
# =====================================================
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    # Quitar acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    # Quitar caracteres especiales
    texto = re.sub(r'[^A-Za-z0-9\s]', '', texto)
    # Quitar espacios duplicados
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# ---------- Convertir todo el texto a minúsculas ----------
columnas_texto = df.select_dtypes(include=['object']).columns
for col in columnas_texto:
    df[col] = df[col].str.lower()


# =====================================================
# TRANSFORM
# =====================================================
# ---------- Rellenar tipo de queja ----------

# - Si tickets_de_soporte == 0 → "sin queja"
df.loc[df['tickets_de_soporte'] == 0, 'tipo_de_queja'] = "sin queja"

# - Si tickets_de_soporte == 0 y tipo_queja es null → "sin queja"
df.loc[(df['tipo_de_queja'].isnull()) & (df['tickets_de_soporte'] == 0), 'tipo_de_queja'] = "sin queja"

# - Si tickets_de_soporte > 0 → "Otra queja"
df.loc[(df['tipo_de_queja'].isnull()) & (df['tickets_de_soporte'] > 0), 'tipo_de_queja'] = "otro tipo de queja"

# Reporte después de la corrección
nulos_final = df['tipo_de_queja'].isnull().sum()

print("\n📊 RESUMEN FINAL DEL ETL")
print(f"Nulos restantes en tipo_de_queja: {nulos_final}")

# ---------- Variables binarias ----------
binarias = ['descuento_aplicado', 'aumento_ultimos_3_meses']

mapa_binario = {
    'si': 1,
    'sí': 1,
    'no': 0,
    '1': 1,
    '0': 0
}

for col in binarias:
    if col in df.columns:
        normalizado = (
            df[col]
            .astype(str)
            .str.strip()
            .str.lower()
        )

        df[col] = normalizado.map(mapa_binario)

        # Valores no reconocidos → 0
        df[col] = df[col].fillna(0).astype(int)

        df[col] = df[col].fillna(0).astype(int)

# ---------- Detectar columnas con más de 2 decimales ----------
columnas_numericas = df.select_dtypes(include=[np.number]).columns
columnas_a_redondear = []

for col in columnas_numericas:
    tiene_muchos_decimales = (
        (df[col].dropna() % 1).round(10)
        .apply(lambda x: len(str(x).split('.')[-1].rstrip('0')) > 2)
        .any()
    )

    if tiene_muchos_decimales:
        columnas_a_redondear.append(col)
        df[col] = df[col].round(2)

print("Columnas redondeadas:", columnas_a_redondear)

# ---------- Limpieza de texto ----------
columnas_texto = df.select_dtypes(include=['object']).columns
for col in columnas_texto:
    df[col] = df[col].apply(limpiar_texto)

# ---------- Validación de rango de puntuación ----------
def escalar(x):
    """
    Convierte valores de rango [-100, 100] a [0, 10]
    """
    return ((x + 100) * 10 / 200)

# ---------- Escalado de puntuación ----------

# Cambia 'puntuacion' si tu columna tiene otro nombre
if 'puntuacion_nps' in df.columns:
  df['puntuacion_nps'] = df['puntuacion_nps'].apply(escalar).round(0)

if 'puntuacion_nps' in df.columns:
    assert df['puntuacion_nps'].between(0, 10).all(), \
        "Existen valores fuera del rango 0–10 en puntuacion_nps"

# ---------- Normalización de categorías ----------
if 'metodo_de_pago' in df.columns:
    mask = df['metodo_de_pago'].str.strip().str.lower() == 'transferencia bancarias'
    df.loc[mask, 'metodo_de_pago'] = 'transferencia bancaria'

if 'canal_de_registro' in df.columns:
    mask = df['canal_de_registro'].str.strip().str.lower() == 'referral'
    df.loc[mask, 'canal_de_registro'] = 'referido'

# =====================================================
# VALIDACIONES DE CALIDAD
# =====================================================

# --- Sin nulos ---
nulos = df.isnull().sum()
nulos = nulos[nulos > 0]

if len(nulos) > 0:
    print("⚠️ Nulos por columna:")
    print(nulos)
else:
    print("No existen valores nulos en el dataset")


# Corrección automática de nulos
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype in ['int64', 'float64']:
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = df[col].fillna('No informado')

# Validar rango permitido (1–5)
df['csates_invalido'] = df['puntuacion_csates'].apply(lambda x: 1 if x < 1 or x > 5 else 0)

# Reporte de inconsistencias
reporte_csates = df[df['csates_invalido'] == 1][['cliente_id', 'puntuacion_csates', 'respuesta_de_la_encuesta']]

if len(reporte_csates) > 0:
    print("⚠️ Registros con puntuacion_csates fuera de rango:")
    print(reporte_csates)
else:
    print("No existen inconsistencias en puntuacion_csates")

# Conteo total
print(f"Total de inconsistencias en CSAT: {df['csates_invalido'].sum()}")

columnas_remover=[
'csates_invalido'
]
df = df.drop(columnas_remover, axis=1, errors='ignore')
print(f"Columnas eliminadas: {', '.join(columnas_remover)}")

# Validación final
assert df.isnull().sum().sum() == 0, "Existen valores nulos en el dataset final"

# --- Binarios válidos ---
for col in binarias:
    if col in df.columns:
        assert set(df[col].dropna().unique()) <= {0, 1}, \
            f"Valores inválidos en {col}"

# --- Sin duplicados ---
assert df.duplicated().sum() == 0, "Existen registros duplicados"

# --- Dataset no vacío ---
assert df.shape[0] > 0, "El dataset final está vacío"

# =====================================================
# LOAD
# =====================================================
df.to_csv(
    'customer_dataset_etl.csv',
    index=False,
    encoding='utf-8'
)

print("========================================")
print("✅ ETL ejecutado correctamente.")



📊 RESUMEN FINAL DEL ETL
Nulos restantes en tipo_de_queja: 0
Columnas redondeadas: ['promedio_conexion', 'tiempo_promedio_de_resolucion', 'tasa_apertura_email']
No existen valores nulos en el dataset
No existen inconsistencias en puntuacion_csates
Total de inconsistencias en CSAT: 0
Columnas eliminadas: csates_invalido
✅ ETL ejecutado correctamente.
